In [ ]:
import os
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

class ImageClassifier:
    def __init__(self, base_path, image_size=(224, 224), batch_size=32, epochs=20, num_classes=3):
        self.base_path = base_path
        self.image_size = image_size
        self.batch_size = batch_size
        self.epochs = epochs
        self.num_classes = num_classes
        self.train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            validation_split=0.2
        )

    def build_model(self):
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=self.image_size + (3,))
        for layer in base_model.layers:
            layer.trainable = False
        
        x = Flatten()(base_model.output)
        x = Dense(256, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(128, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        predictions = Dense(self.num_classes, activation='softmax')(x)
        
        model = Model(inputs=base_model.input, outputs=predictions)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
        return model

    def train_and_evaluate(self):
        train_generator = self.train_datagen.flow_from_directory(
            self.base_path,
            target_size=self.image_size,
            batch_size=self.batch_size,
            class_mode='categorical',
            subset='training'
        )

        validation_generator = self.train_datagen.flow_from_directory(
            self.base_path,
            target_size=self.image_size,
            batch_size=self.batch_size,
            class_mode='categorical',
            subset='validation'
        )

        model = self.build_model()

        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

        model.fit(
            train_generator,
            epochs=self.epochs,
            validation_data=validation_generator,
            callbacks=[early_stopping, reduce_lr]
        )

        # Evaluate the model
        test_loss, test_accuracy = model.evaluate(validation_generator)
        print(f"Test accuracy: {test_accuracy:.2f}")

if __name__ == "__main__":
    base_path = "./processed_data"
    classifier = ImageClassifier(base_path)
    classifier.train_and_evaluate()
